## Notebook to figure out mesh generation and remeshing
Standard Python 3.12.3 kernel, not conda.

In [1]:
# importing the relevant packages
import pyvista as pv
import scipy.spatial as sp
import meshio
import os
import matplotlib.pyplot as plt
import numpy as np
import pygmsh as pg
import jax.numpy as jnp
from units_constants import ConversionFactors as uc

### Pygmsh meshes

In [ ]:
with pg.geo.Geometry() as geom:
    # Define points of a cube
    resolution = 0.1
    p0 = geom.add_point([0, 0, 0], resolution)
    p1 = geom.add_point([1, 0, 0], resolution)
    p2 = geom.add_point([1, 1, 0], resolution)
    p3 = geom.add_point([0, 1, 0], resolution)
    p4 = geom.add_point([0, 0, 1], resolution)
    p5 = geom.add_point([1, 0, 1], resolution)
    p6 = geom.add_point([1, 1, 1], resolution)
    p7 = geom.add_point([0, 1, 1], resolution)

    # Define lines (12 edges of a cube)
    l01 = geom.add_line(p0, p1)
    l12 = geom.add_line(p1, p2)
    l23 = geom.add_line(p2, p3)
    l30 = geom.add_line(p3, p0)

    l45 = geom.add_line(p4, p5)
    l56 = geom.add_line(p5, p6)
    l67 = geom.add_line(p6, p7)
    l74 = geom.add_line(p7, p4)

    l04 = geom.add_line(p0, p4)
    l15 = geom.add_line(p1, p5)
    l26 = geom.add_line(p2, p6)
    l37 = geom.add_line(p3, p7)

    # Define curve loops and surfaces (6 faces)
    bottom = geom.add_surface(geom.add_curve_loop([l01, l12, l23, l30]))
    top    = geom.add_surface(geom.add_curve_loop([l45, l56, l67, l74]))
    side1  = geom.add_surface(geom.add_curve_loop([l01, l15, -l45, -l04]))
    side2  = geom.add_surface(geom.add_curve_loop([l12, l26, -l56, -l15]))
    side3  = geom.add_surface(geom.add_curve_loop([l23, l37, -l67, -l26]))
    side4  = geom.add_surface(geom.add_curve_loop([l30, l04, -l74, -l37]))

    # Create surface loop and volume
    surface_loop = geom.add_surface_loop([side4, top, side1, bottom, side3, side2]) # order doesn't matter here, it does for the stuff before
    volume = geom.add_volume(surface_loop)

    # Generate 3D mesh
    mesh = geom.generate_mesh()

    # Optional: Save
    mesh.write("box.stl")

mesh = pv.read("box.stl")

#pv.plot(mesh, show_edges = True)

KeyError: dtype('<u8')

### Delaunay triangulation/tetrahedron-ization

In [ ]:
points = np.array([
    [0.0, 0.0, 0.0],
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [1.0, 1.0, 0.0],
    [2.0, 0.0, 0.0],
    [2.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [1.0, 0.0, 1.0],
    [0.0, 1.0, 1.0],
    [1.0, 1.0, 1.0],
    [2.0, 0.0, 1.0],
    [2.0, 1.0, 1.0],])

tetra = sp.Delaunay(points)
cells = [("tetra", tetra.simplices)]
print(len(points))
print(len(cells[0][1]))

mesh = meshio.Mesh(
    points[:6],
    cells[:6],
)
meshio.write_points_cells("fooDel.vtk", points, cells)
Del_mesh = pv.read("fooDel.vtk")
mesh

# plotting
meshio_plot = pv.Plotter()
meshio_plot.add_mesh(Del_mesh, show_edges = True)
meshio_plot.show_axes()
meshio_plot.show()

    

12
12


Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd3c13c1d0_16&reconnect=auto" class="pyvi…

Vertex = shape vertices, not the cells \
LInes = lines you can see on the volume boundary \
Triangles = triangles you can see on  \
Tetras = total tetras?

Points = points inside \
Cells = cells total \

### Pygmsh mesh creation
I followed Le Chat first but I get an error, so best to follow Dokken's tutorial instead? 

05/04/2025 update: the error was that the Le Chat code had a bug and was giving outdated syntax. Got the correct syntax from pypi: [link](https://pypi.org/project/pygmsh/) 

11/04/2025 update: don't use Le Chat for explicit coding, use chatGPT for that.

In [ ]:
# Step 1: generate the mesh with pygmsh
with pg.occ.Geometry() as geom:
    geom.add_ellipsoid([.0, .0, .0], [1., 1., 1.], mesh_size = .5) # sphere = ellipsoid with a = b = c
    mesh = geom.generate_mesh()

# a mesh contains all element types, so all lines, vertices, surfaces and cells
# we only add the elements to the meshio mesh that we are interested in, in this case the tetrahedrons
points = mesh.points
mesh

tetra_cells = [cell.data for cell in mesh.cells if cell.type == "tetra"]
cells = [("tetra", tetra_cells[0])]

# Compute 1/r falloff at cell centers
center = np.mean(points, axis=0)
cell_centers = np.mean(points[tetra_cells[0]], axis=1)
distances = np.linalg.norm(cell_centers - center, axis=1)
value = 1.0 / np.where(distances == 0, 1e-12, distances)

# Create meshio mesh with cell data
meshio_mesh = meshio.Mesh(
    points=points,
    cells=cells,
    cell_data={"value": [value]}
)
# print(meshio_mesh.cells)
# print(len(meshio_mesh.points))
# write meshio mesh to a vtk file
meshio.write("sphere_coarse.vtk", meshio_mesh)

sphere_mesh = pv.read("sphere_coarse.vtk")
sphere_mesh
pv.plot(sphere_mesh)
sphere_slice = sphere_mesh.slice(normal=[1,1,1])

# plotting
pg_plot = pv.Plotter(shape=(1,2))
pg_plot.add_mesh(sphere_mesh, show_edges = True, scalars = "value")
pg_plot.subplot(0, 1)
pg_plot.add_mesh(sphere_slice, show_edges = True, scalars = "value")
pg_plot.show_axes()
pg_plot.show()

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd24da6570_17&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd3c113c50_18&reconnect=auto" class="pyvi…

In [ ]:
# making another sphere
with pg.geo.Geometry() as geom:
    geom.add_ball([.0, .0, .0], 1., mesh_size = .1) # sphere = ellipsoid with a = b = c, or ball
    mesh = geom.generate_mesh()
points = mesh.points
#print(c_points) 
cells = mesh.cells

# return value is always a meshio mesh.
mesh.write("sphere_fine.vtk")

sphere_mesh = pv.read("sphere_fine.vtk")

# plotting
pg_plot = pv.Plotter()
pg_plot.add_mesh(sphere_mesh, show_edges = True)
pg_plot.show_axes()
pg_plot.show()
    


Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd24d88f20_19&reconnect=auto" class="pyvi…

Mesh refinement from PyPI example. Doesn't work yet. 

In [ ]:
# Step 1: generate the mesh with pygmsh
with pg.occ.Geometry() as geom:
    geom.add_box([.0, .0, .0], [1., 1., 1.], mesh_size = .2) # sphere = ellipsoid with a = b = c
    mesh = geom.generate_mesh()

# a mesh contains all element types, so all lines, vertices, surfaces and cells
# we only add the elements to the meshio mesh that we are interested in, in this case the tetrahedrons
points = mesh.points
tetra_cells = [cell.data for cell in mesh.cells if cell.type == "tetra"]
cells = [("tetra", tetra_cells[0])]

# Compute 1/r falloff at cell centers
center = np.mean(points, axis=0)
cell_centers = np.mean(points[tetra_cells[0]], axis=1)
distances = np.linalg.norm(cell_centers - center, axis=1)
value = 1.0 / np.where(distances == 0, 1e-12, distances)

# Create meshio mesh with cell data
meshio_mesh = meshio.Mesh(
    points=points,
    cells=cells,
    cell_data={"value": [value]}
)

# write meshio mesh to a vtk file
meshio.write("cube_coarse.vtk", meshio_mesh)

cube_mesh = pv.read("cube_coarse.vtk")

# plotting
pg_plot = pv.Plotter()
pg_plot.add_mesh(cube_mesh, show_edges = True, scalars = "value")
pg_plot.show_axes()
pg_plot.show()

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1ccf81e68a0_20&reconnect=auto" class="pyvi…

In [ ]:
# making another cube
with pg.occ.Geometry() as geom:
    geom.add_box([.0, .0, .0], [1., 1., 1.], mesh_size = .05) 
    mesh = geom.generate_mesh()
points = mesh.points
#print(c_points) 
cells = mesh.cells

# return value is always a meshio mesh.
mesh.write("cube_fine.vtk")

cube_mesh = pv.read("cube_fine.vtk")

# plotting
pg_plot = pv.Plotter()
pg_plot.add_mesh(cube_mesh, show_edges = True)
pg_plot.show_axes()
pg_plot.show()
    

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1ccffe42b70_21&reconnect=auto" class="pyvi…

Gmsh has much more control and customizability, and more options for curves and lines. But may be too intensive for a first prototype. I can always switch to it later.

In [ ]:
# testing gmsh instead of pygmsh
gm.initialize()
gm.model.add("Test box")
ms = 0.5


## Old, return later
### Test run with the warped meshes from session 12
02/04/2025: Of course this doesn't work as expected because I am supplying all the points, where I should only supply the outer shell, then create new nodes inside."\\

Plan of attack:
1. Obtain indiced of nodes on the outside of the mesh (surface nodes) by using `np.isclose(points[X], 0, atol = 1e-5)` on an undeformed mesh. Make sure to remove any double numbers, which can happen if a node satisfies both the x and y condition for example. 
    - This is a one time step. 
2. Use those indices to obtain the surface nodes on a warped mesh. 
    - From here on out we repeat at every remesh.
3. Apply Delaunay triangulation and see what happens. 

In [ ]:
# # read in the data. Accepts .vtu/.vtk
# sesh_num = 12
# folder = "field"
# compr = 80 # deformation percentage (01 = 0.1%, 10 = 10%, etc.)
# N_row = 10
# warpfactor = 1
# solver = "umfpack"
# E_val = 1.0
# rho_center = 0.0005*2.65
# steps = 80

# scalar = 'density (g/cm^3)'

# # laptop
# parent_dir12 = f'C:/Users/spaan/Thesis/TwelfthSesh_25-03-2025/sil-density-model-{sesh_num}_{compr}_{solver}_solver_{rho_center}_{E_val}rho_{N_row**3}cells_{steps}steps/vtk/'
# parent_dir13 = f'C:/Users/spaan/Thesis/ThirteenthSesh_31-03-2025/sil-density-model-{sesh_num}_{compr}_{solver}_solver_{rho_center}_{E_val}rho_{N_row**3}cells_{steps}steps/vtk/'
# parent_dir14 = f'C:/Users/spaan/Thesis/FourteenthSesh_31-03-2025/sil-density-model-{sesh_num}_{compr}_{solver}_solver_{rho_center}_{E_val}rho_{N_row**3}cells_{steps}steps/vtk/'


# # home pc
# # parent_dir_centerfield = f'C:/Users/Lena/UvA/Thesis/Fluffy_Aggregates/density-model-{folder}_{compr}_{solver}_{rho_center}_{E_val}/vtk/'


# parent_dir = parent_dir12
# print(parent_dir)

# meshes = []
# warped_meshes = []
# for subdir, dirs, files in os.walk(parent_dir):
#     for file in files:
#         fdir = parent_dir + file

#         pv_mesh = pv.read(fdir)
#         meshes.append(pv_mesh)
#     n = len(files) # number of files = number of compression steps

# print(f"Reading is done! {n} files")

# # keep in mind that the factor of course works exponentially, so don't use the warp except for the final result, or scale by smt. else.
# for mesh in meshes:
#     warped_mesh = mesh.warp_by_vector('sol', factor = warpfactor)
#     warped_meshes.append(warped_mesh)


# low_mesh_num = 2
# mid_mesh_num = int(n/2)
# hi_mesh_num = n-3
# zero_frame = 0
# wframe = mid_mesh_num

# initmesh_points = meshes[zero_frame].points
# warped_mesh_points = warped_meshes[mid_mesh_num].points
# N = len(initmesh_points)
# surf_points_idx = []


# def issurf(point, val):
#     return np.isclose(point, val, atol = 1e-2)


# # boolean arrays on which node lies on the specified surface
# isbottom = issurf(initmesh_points[:,2], 0)
# istop = issurf(initmesh_points[:,2], 1)
# isside2 = issurf(initmesh_points[:,0], 0)
# isside4 = issurf(initmesh_points[:,0], 1)
# isside3 = issurf(initmesh_points[:,1], 0)
# isside5 = issurf(initmesh_points[:,1], 1)

# issurface = [isbottom, isside2, isside3, isside4, isside5, istop]

# # retrieving the indices of the surface nodes
# for surf in issurface:
#     true_surf = list(filter(lambda i: surf[i], range(len(surf))))
#     surf_points_idx.append(true_surf)


# # a very roundabout way of turning the 2D list into a flattened array, removing the duplicate elements, 
# # and sorting it into surf_points
# surf_points_idx = set(np.array(surf_points_idx).flatten())
# surf_points_idx = np.array(list(surf_points_idx))
# #print(surf_points_idx)


# # applying the deformation to the points so that I get the outside
# surf_points = warped_mesh_points[surf_points_idx]
# N_surf = len(surf_points)
# #print(surf_points)


# # scattering N - len(surface_points) throughout the new mesh
# xmin, xmax = surf_points[:,0].min(), surf_points[:,0].max()
# ymin, ymax = surf_points[:,1].min(), surf_points[:,1].max()
# zmin, zmax = surf_points[:,2].min(), surf_points[:,2].max()

# np.random.seed(0) # reproducibility
# offset = 0.95 # so that we don't generate points on the surface, only inside
# x_coords = offset*(xmax - xmin)*np.random.rand(N-N_surf) + offset*xmin
# y_coords = offset*(ymax - ymin)*np.random.rand(N-N_surf) + offset*ymin
# z_coords = offset*(zmax - zmin)*np.random.rand(N-N_surf) + offset*zmin

# print("First point: ", x_coords[0], y_coords[0], z_coords[0])
# new_inpoints = np.stack((x_coords, y_coords, z_coords), axis = -1)
# print("First point after stacking: ", new_inpoints[0])
# new_points = np.concatenate((surf_points, new_inpoints))
# print(len(new_points), new_points.shape)


# # Delaunay triangulation on the surface points
# remeshed = sp.Delaunay(new_points)
# remeshed_cells = [("tetra", remeshed.simplices)]
# remeshed_points = remeshed.points
# print(remeshed_cells[0][1][0])

# new_mesh = meshio.Mesh(remeshed_points, remeshed_cells)
# meshio.write_points_cells(f"frame{wframe}_remeshed.vtk", remeshed_points, remeshed_cells)
# remeshed_Delmesh = pv.read(f"frame{wframe}_remeshed.vtk")

# # plotting
# meshio_plot = pv.Plotter()
# meshio_plot.add_mesh(remeshed_Delmesh, show_edges = True)
# meshio_plot.show()

## Warped Mesh
Generating a new mesh from the old one. Taking frame 20 from sesh 14 as a test.

In [ ]:
# Sim meshes
init_mesh = pv.read("FourteenthSesh_31-03-2025/sil-density-model-14_80_umfpack_solver_0.0007949999999999999_1.0rho_1000cells_80steps/vtk/sol_001.vtu")
init_mesh = init_mesh.warp_by_vector('sol', factor = 1)
mesh20_sesh14 = pv.read("FourteenthSesh_31-03-2025/sil-density-model-14_80_umfpack_solver_0.0007949999999999999_1.0rho_1000cells_80steps/vtk/sol_020.vtu")
mesh20_sesh14 = mesh20_sesh14.warp_by_vector('sol', factor = 1)
mesh79_sesh14 = pv.read("FourteenthSesh_31-03-2025/sil-density-model-14_80_umfpack_solver_0.0007949999999999999_1.0rho_1000cells_80steps/vtk/sol_079.vtu")
mesh79_sesh14 = mesh79_sesh14.warp_by_vector('sol', factor = 1)
sim_mesh = mesh79_sesh14

print("Reading is done!")

Reading is done!


Checking the mesh data

In [ ]:
sim_mesh

UnstructuredGrid (0x1ccbec51ea0)
  N Cells:    1000
  N Points:   1331
  X Bounds:   -3.907e-02, 1.039e+00
  Y Bounds:   -3.907e-02, 1.039e+00
  Z Bounds:   0.000e+00, 2.000e-01
  N Arrays:   4

In [ ]:
#pv.plot(sim_mesh, show_edges = True, scalars = 'density (g/cm^3)')
pv.plot(sim_mesh, scalars = "density (g/cm^3)")
print(len(sim_mesh.points))

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd3c13c350_22&reconnect=auto" class="pyvi…

1331


### Interpolation - A big big note
A big big note on constructing meshes. The documentation of gmsh (not pygmsh) has links for a lot of functions to coding examples, in which they construct meshes. That is going to be hella important for this bit - you don't want to fuck this up by cutting corners and thinking you *approximately* know it. Just be methodical, and read. \n

If you get stuck with the surfaces, read this stackoverflow post: https://stackoverflow.com/questions/78074617/pygmsh-how-to-create-3d-mesh-from-2d-cross-sections 
Or any `add_volume`/`addVolume` question.

In [ ]:
# 0. Sort the nodes into their respective surfaces by index
# Wait maybe this isn't necessary, I can also extract edges :P
init_surf = init_mesh.extract_surface

In [ ]:
# 1. Extract old mesh surface - thoroughly read the documentation for this function! This may play a role in the assigning-points-to-data. 
# Also look up connectivity and don't forget to add points to the surface, to increase resolution
surf = sim_mesh.extract_surface(nonlinear_subdivision=5)
edges = sim_mesh.extract_feature_edges()
surf.points # xyz coordinates for each point
surf.faces # face tags, so not coordinates
print(surf.faces)

# extract point indices per edge
# edge_points contains all the points grouped by line segment
edge_lines =  edges.lines.reshape(-1, 3) # each row: [2, p1, p2]
edge_points = []
for _, p1_idx, p2_idx, in edge_lines:
    edge_points.append((edges.points[p1_idx], edges.points[p2_idx]))

# pe = pv.Plotter()
# pe.add_mesh(surf.points)
# pe.show()

#print(edge_points[0][0])


[  4   0   1 ... 181  48 184]


Option 1, manual

In [ ]:
# 2. Generate new mesh with a certain resolution
# Follow the Dokken tutorial for this, to create a mesh from points and lines. See par. 6.6 (geo) or 6.8 (occ) from the gmsh documentation for evrything. 
# that can be added (incl. splines).
# Play around with resolutions. Look into the measure of resolution - perhaps it's a percentage? Mehs module, 1.2 in 
# the documentation:https://gmsh.info/doc/texinfo/gmsh.html#Specifying-mesh-element-sizes


resolution = 0.5
# minimum and maximum values for each axis
xmin, xmax = np.min(surf.points[:,0]), np.max(surf.points[:,0])
ymin, ymax = np.min(surf.points[:,1]), np.max(surf.points[:,1])
zmin, zmax = np.min(surf.points[:,2]), np.max(surf.points[:,2])

# sort points per edge first, and then in coordinate order
# see obsidian/notebook


with pg.occ.Geometry() as geom:
    # points contains all pygmsh points (not fixed yet)
    # lines contains all pygmsh lines
    points = []
    lines = []

    # draw lines between points
    for i, point_set in enumerate(edge_points):
        # point_set is a pair of points (p1, p2) that are connected with a line
        p1 = geom.add_point(point_set[0], mesh_size = resolution)
        p2 = geom.add_point(point_set[1], mesh_size = resolution)
        l = geom.add_line(p1, p2) 

        lines.append(l)

    print(l.points[0].x) # the coordinates of the first point of the line
    #isbottom = np.isclose()
    # # create line loop
    # line_loop = geom.add_curve_loop(lines)

    # # create a surface
    # surface = geom.add_ruled_surface(line_loop)
    
    
    mesh = geom.generate_mesh()


pv.plot(mesh)


[-0.03637468  1.03637468  0.18466594]


Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd4d7f8bc0_23&reconnect=auto" class="pyvi…

In [ ]:
# tried to read in a remeshed gmsh file with pyvista but no.
# FourteenthSesh_31-03-2025\sil-density-model-14_80_umfpack_solver_0.0007949999999999999_1.0rho_1000cells_80steps\box_refined.vtu

In [ ]:
# # 3. Plot new mesh to check
# # pv.plot(new_mesh)

In [ ]:
# Trying Daniel's thing for getting the connectivity of a concave surface
surface_points = surf.points

# 1. Project surface onto unit sphere
CM = np.sum(surface_points, axis=0)/len(surface_points)
surface_points = surface_points - CM
norm = np.linalg.norm(surface_points, axis = 1)
surface_points = surface_points/norm.reshape(-1, 1)
norm_new = np.linalg.norm(surface_points, axis = 1)
print(norm_new[0]) # norm check



pv.plot(surface_points)

# 2. 



1.0


Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd4d802ba0_24&reconnect=auto" class="pyvi…

In [ ]:
with pg.geo.Geometry() as geom:
    ball = geom.add_ellipsoid([0,0,0],[5, 5, 5], mesh_size = 0.5, with_volume = False)
    ball_mesh = geom.generate_mesh()

    ball_mesh.points[:, 2] += 0.5 # this should move the entire ball up by 0.5. And it does!


    surface_loop = ball.surface_loop
    volume = geom.add_volume(surface_loop)

    moved_mesh = ball_mesh
    new_mesh = geom.generate_mesh()

The moved mesh has no volume elements, as is expected.

In [ ]:
moved_mesh

<meshio mesh object>
  Number of points: 1662
  Number of cells:
    line: 192
    triangle: 3074
    vertex: 7

The new_mesh does have volume elements, because we added those to the ball geometry.

In [ ]:
new_mesh

<meshio mesh object>
  Number of points: 3908
  Number of cells:
    line: 192
    triangle: 3074
    tetra: 18543
    vertex: 7

Plotting them together shows this: only one mesh has moved, the other remains, even though we generated them from the same geometric object. 

In [ ]:
ball_plot = pv.Plotter()
ball_plot.add_mesh(moved_mesh, opacity=0.5)
ball_plot.add_mesh(new_mesh, color = 'green')
ball_plot.show()

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1ccd0829f70_25&reconnect=auto" class="pyvi…

Can I convert a `.vtk` file to `.msh`? Oh thank goodness yes I can, see below.

In [ ]:
init_mesh_vtu = meshio.read("FourteenthSesh_31-03-2025/sil-density-model-14_80_umfpack_solver_0.0007949999999999999_1.0rho_1000cells_80steps/vtk/sol_001.vtu")
sphere_vtk = meshio.read("sphere_coarse.vtk")
#meshio.write("sphere_coarse.msh", sphere_vtk, file_format="gmsh")
meshio.write("init_mesh.msh", init_mesh_vtu, file_format = "gmsh")

sphere_msh = pv.read("sphere_coarse.msh")
#sphere_msh["value"]
#pv.plot(sphere_msh)

In [ ]:
d = np.arange(10)
y = np.sin(d)



with open()

## Different tries for remeshing using other libraries - spare time project
### Pyacvd for remeshing?

In [ ]:
import pyacvd
# cow = pv.examples.download_cow()
# cow = cow.triangulate()
# clus = pyacvd.Clustering(cow)
# clus.subdivide(3)
# clus.cluster(20000)
# remesh = clus.create_mesh()
# remesh.plot(show_edges = True)

Widget(value='<iframe src="http://localhost:56079/index.html?ui=P_0x1cd24d8be90_15&reconnect=auto" class="pyvi…

In [ ]:
# # try this later
# tri_mesh = sim_mesh.triangulate()
# tri_mesh_polydata = pv.PolyData(tri_mesh.points)
# #pv.plot(tri_mesh)
# clus = pyacvd.Clustering(tri_mesh_polydata)
# clus.subdivide(3)
# clus.cluster(20000)
# remesh = clus.create_mesh()
# remesh.plot(show_edges = True)

ValueError: Input mesh must be composed of all triangles. Hint: `mesh.triangulate` first.